In [1]:
!pip install "sagemaker>=2.199.0" gradio transformers --upgrade --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.6.0 requires tensorboard-data-server<0.7.0,>=0.6.0, which is not installed.
mlflow 2.3.1 requires docker<7,>=4.0.0, but you have docker 7.1.0 which is incompatible.
tensorboard 2.6.0 requires google-auth<2,>=1.6.3, but you have google-auth 2.6.0 which is incompatible.


In [2]:
import sagemaker
import boto3

region = 'eu-west-2'

boto_session = boto3.Session(region_name=region)
sess = sagemaker.Session(boto_session=boto_session)
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(boto_session=boto_session, default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {sess.boto_region_name}")

bedrock_runtime = boto3.client('bedrock-runtime', region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\karol\AppData\Local\sagemaker\sagemaker\config.yaml


Couldn't call 'get_role' to get Role ARN from role name superUser to get Role path.


sagemaker role arn: arn:aws:iam::767427092061:role/sagemaker_execution_role
sagemaker session region: eu-west-2


In [25]:
user_prompt = "Znasz język polski? napisz coś o sienkiewiczu?"

prompt = f"""<s>[INST] <<SYS>>
Jesteś pomocnym, pełnym szacunku i uczciwym asystentem. Zawsze odpowiadaj tak pomocnie, jak to możliwe, jednocześnie zachowując bezpieczeństwo. Twoje odpowiedzi nie powinny zawierać żadnych treści szkodliwych, nieetycznych, rasistowskich, seksistowskich, toksycznych, niebezpiecznych ani nielegalnych. Upewnij się, że twoje odpowiedzi są społecznie bezstronne i pozytywne.


Jeśli pytanie nie ma sensu lub nie jest logicznie spójne, wyjaśnij dlaczego, zamiast udzielać niepoprawnej odpowiedzi. Jeśli nie znasz odpowiedzi na pytanie, nie podawaj fałszywych informacji.
<</SYS>>
{user_prompt} [/INST]
"""


In [26]:
prompt

'<s>[INST] <<SYS>>\nJesteś pomocnym, pełnym szacunku i uczciwym asystentem. Zawsze odpowiadaj tak pomocnie, jak to możliwe, jednocześnie zachowując bezpieczeństwo. Twoje odpowiedzi nie powinny zawierać żadnych treści szkodliwych, nieetycznych, rasistowskich, seksistowskich, toksycznych, niebezpiecznych ani nielegalnych. Upewnij się, że twoje odpowiedzi są społecznie bezstronne i pozytywne.\n\n\nJeśli pytanie nie ma sensu lub nie jest logicznie spójne, wyjaśnij dlaczego, zamiast udzielać niepoprawnej odpowiedzi. Jeśli nie znasz odpowiedzi na pytanie, nie podawaj fałszywych informacji.\n<</SYS>>\nZnasz język polski? napisz coś o sienkiewiczu? [/INST]\n'

In [27]:
import json

kwargs = {
    "modelId": "meta.llama3-70b-instruct-v1:0",
    "contentType": "application/json",
    "accept": "*/*",
    "body": json.dumps(
        {
            "prompt": prompt,
            "max_gen_len": 512,
            "temperature": 0.5,
            "top_p": 0.9,
        }
    )
}

In [28]:
response = bedrock_runtime.invoke_model(**kwargs)

In [29]:
response_body = json.loads(response.get('body').read())

In [30]:
print(json.dumps(response_body, indent=4))

{
    "generation": "Henryk Sienkiewicz to jeden z najwi\u0119kszych pisarzy polskich, laureat Nagrody Nobla w dziedzinie literatury w 1905 roku. Urodzi\u0142 si\u0119 5 maja 1846 roku w Woli Okrzejskiej, a zmar\u0142 15 listopada 1916 roku w Vevey w Szwajcarii.\n\nSienkiewicz jest autorem wielu znanych powie\u015bci historycznych, takich jak \"Quo Vadis\", \"Krzy\u017cacy\", \"Pan Wo\u0142odyjowski\" i \"Na jaki\u015b czas\". Jego tw\u00f3rczo\u015b\u0107 by\u0142a inspiracj\u0105 dla wielu innych pisarzy i artyst\u00f3w.\n\nW swoich utworach Sienkiewicz cz\u0119sto podejmowa\u0142 tematyk\u0119 historyczn\u0105, religijn\u0105 i patriotyczn\u0105. Jego styl pisarski charakteryzuje si\u0119 bogactwem j\u0119zyka, dynamizmem akcji i g\u0142\u0119bi\u0105 psychologiczn\u0105 postaci.\n\nWarto wspomnie\u0107, \u017ce Sienkiewicz by\u0142 tak\u017ce publicyst\u0105 i redaktorem, a jego artyku\u0142y i eseje by\u0142y publikowane w r\u00f3\u017cnych czasopismach. Dzi\u015b jest uwa\u017can

In [31]:
print(response_body['generation'])

Henryk Sienkiewicz to jeden z największych pisarzy polskich, laureat Nagrody Nobla w dziedzinie literatury w 1905 roku. Urodził się 5 maja 1846 roku w Woli Okrzejskiej, a zmarł 15 listopada 1916 roku w Vevey w Szwajcarii.

Sienkiewicz jest autorem wielu znanych powieści historycznych, takich jak "Quo Vadis", "Krzyżacy", "Pan Wołodyjowski" i "Na jakiś czas". Jego twórczość była inspiracją dla wielu innych pisarzy i artystów.

W swoich utworach Sienkiewicz często podejmował tematykę historyczną, religijną i patriotyczną. Jego styl pisarski charakteryzuje się bogactwem języka, dynamizmem akcji i głębią psychologiczną postaci.

Warto wspomnieć, że Sienkiewicz był także publicystą i redaktorem, a jego artykuły i eseje były publikowane w różnych czasopismach. Dziś jest uważany za jednego z największych pisarzy polskich wszech czasów.


In [23]:
print(response)

{'ResponseMetadata': {'RequestId': '6deb8919-360c-412c-95e9-1b27b2c409b1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 08 Sep 2024 11:50:03 GMT', 'content-type': 'application/json', 'content-length': '89', 'connection': 'keep-alive', 'x-amzn-requestid': '6deb8919-360c-412c-95e9-1b27b2c409b1', 'x-amzn-bedrock-invocation-latency': '173', 'x-amzn-bedrock-output-token-count': '1', 'x-amzn-bedrock-input-token-count': '23'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0x000001C4C1DEBE80>}


In [1]:
from langchain_core.tools import StructuredTool, tool
from langchain_core.utils.function_calling import convert_to_openai_tool
import types


def calculate_retail_margin(x):
    pass


try:
    exec('''def oblicz_marze_detaliczna(cena_hurtowa):
    """
    Oblicza urzędową marżę detaliczną na podstawie ceny hurtowej leku, środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu medycznego.

    Argumenty:
    cena_hurtowa (float): Cena hurtowa w złotych.

    Zwraca:
    float: Obliczona marża detaliczna w złotych.

    Uwagi:
    Funkcja obsługuje różne przedziały cenowe zgodnie z dostarczoną tabelą marży detalicznej.
    """
    if cena_hurtowa <= 5.00:
        marza = 0.40 * cena_hurtowa
    elif 5.01 <= cena_hurtowa <= 10.00:
        marza = 2 + 0.30 * (cena_hurtowa - 5.00)
    elif 10.01 <= cena_hurtowa <= 20.00:
        marza = 3.50 + 0.20 * (cena_hurtowa - 10.00)
    elif 20.01 <= cena_hurtowa <= 40.00:
        marza = 5.50 + 0.15 * (cena_hurtowa - 20.00)
    elif 40.01 <= cena_hurtowa <= 80.00:
        marza = 8.50 + 0.10 * (cena_hurtowa - 40.00)
    elif 80.01 <= cena_hurtowa <= 160.00:
        marza = 12.50 + 0.05 * (cena_hurtowa - 80.00)
    elif 160.01 <= cena_hurtowa <= 320.00:
        marza = 16.50 + 0.025 * (cena_hurtowa - 160.00)
    elif 320.01 <= cena_hurtowa <= 640.00:
        marza = 20.50 + 0.025 * (cena_hurtowa - 320.00)
    elif 640.01 <= cena_hurtowa <= 1280.00:
        marza = 28.50 + 0.025 * (cena_hurtowa - 640.00)
    else:
        marza = 44.50 + 0.0125 * (cena_hurtowa - 1280.00)
    return marza
    ''')
except SyntaxError as e:
    print(f"SyntaxError: {e}")
except Exception as e:
    print(f"Error: {e}")
function_name = "oblicz_marze_detaliczna"
calculate_margin = locals()[function_name]

price = 750.00
margin2 = calculate_margin(price)

print(f"The margin for {price} zł is {margin2} zł.")

calculate_retail_margin = tool(calculate_margin)

# structured_tool = StructuredTool.from_function(func=calculate_retail_margin)

# print(structured_tool.invoke({"x": price}))
#
# tools = []
#
# tools.append(calculate_retail_margin)
#
# print(calculate_retail_margin)
# print(f"The margin for {price} zł is {margin2} zł.")
l = convert_to_openai_tool(calculate_retail_margin)
l

The margin for 750.0 zł is 31.25 zł.


{'type': 'function',
 'function': {'name': 'oblicz_marze_detaliczna',
  'description': 'Oblicza urzędową marżę detaliczną na podstawie ceny hurtowej leku, środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu medycznego.\n\nArgumenty:\ncena_hurtowa (float): Cena hurtowa w złotych.\n\nZwraca:\nfloat: Obliczona marża detaliczna w złotych.\n\nUwagi:\nFunkcja obsługuje różne przedziały cenowe zgodnie z dostarczoną tabelą marży detalicznej.',
  'parameters': {'type': 'object',
   'properties': {'cena_hurtowa': {}},
   'required': ['cena_hurtowa']}}}

In [33]:
!pip install langchain-aws

     ---------------------------------------- 0.0/82.8 kB ? eta -:--:--
     -------------- ------------------------- 30.7/82.8 kB 1.4 MB/s eta 0:00:01
     -------------------------------- ----- 71.7/82.8 kB 787.7 kB/s eta 0:00:01
     -------------------------------------- 82.8/82.8 kB 772.4 kB/s eta 0:00:00


In [80]:
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_aws import ChatBedrock
from langchain_community.llms import SagemakerEndpoint

llm = ChatBedrock(
    model_id="meta.llama3-70b-instruct-v1:0",
    client=bedrock_runtime,
    region_name=region,
    model_kwargs=dict(temperature=0),
    # other params...
)

l = convert_to_openai_tool(calculate_retail_margin)

l

{'type': 'function',
 'function': {'name': 'oblicz_marze_detaliczna',
  'description': 'Funkcja oblicza urzędową marżę detaliczną na podstawie podanej ceny hurtowej leku, środka spożywczego specjalnego przeznaczenia żywieniowego lub wyrobu medycznego.\n\nArgumenty:\ncena_hurtowa (float): Cena hurtowa produktu.\n\nZwraca:\nfloat: Obliczona marża detaliczna.',
  'parameters': {'type': 'object',
   'properties': {'cena_hurtowa': {}},
   'required': ['cena_hurtowa']}}}

In [88]:
from langchain_core.prompts import ChatPromptTemplate
import json

k = '''{
    "type": "function",
    "function": {
        "name": "get_current_conditions",
        "description": "Get the current weather conditions for a specific location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g., San Francisco, CA"
                },
                "unit": {
                    "type": "string",
                    "enum": ["Celsius", "Fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the user's location."
                }
            },
            "required": ["location", "unit"]
        }
    }
}'''

messages = [
    (
        "system",
        '''When you receive a tool call response, use the output to format an answer to the original user question.You are a helpful assistant with tool calling capabilities.'''
    ),
    ("human", ''' Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.
    Respond in the format {{"name": function name, "parameters": dictionary of argument name and its value}}.
    Do not use variables.
    {function}
    {function_2}
    Question: {input}'''

     ),
]

print(messages)

prompt_bedrock = ChatPromptTemplate.from_messages(messages)

bedrock_chain = prompt_bedrock | llm

ai_msg = bedrock_chain.invoke(
    {"input": "Jaką marżę detaliczną należy zastosować dla leku, którego cena hurtowa wynosi 750 zł?",
     "function": str(l),
     "function_2": k
     }
)

messages_for_return_function = [
    (
        "system",
        f'''
        You are a helpful assistant with tool calling capabilities. When you receive a tool call response, use the output to format an answer to the orginal use question.'''
    ),
    ("human", ''' Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.
    Respond in the format {{"name": function name, "parameters": dictionary of argument name and its value}}.
    Do not use variables.
    {function}
    Question: {input}'''
     ),
    ("ai", "{ai_content} Now here is the result of called function <|start_header_id|>ipython<|end_header_id|> {{'output':' "+str(margin2) +"'}}  Now using full sentences using polish language answer the question. Pay attention to the correctness of the answer like units "),


]

print(ai_msg)

prompt_becdrock_return_function = ChatPromptTemplate.from_messages(messages_for_return_function)

bedrock_chain_return_function = prompt_becdrock_return_function | llm

ai_msg_return_function = bedrock_chain_return_function.invoke(
    {"input": "Jaką marżę detaliczną należy zastosować dla leku, którego cena hurtowa wynosi 750 zł?",

     "function": str(l),
    "ai_content": ai_msg.content
     }
)

print(ai_msg_return_function)

# ai_msg_2 = bedrock_chain.invoke(
#     {"input": "Jaka jest pogoda w Warszawie?",
#      "function": str(l),
#      "function_2": k
#      }
# )
#
# print(ai_msg_2)

[('system', 'When you receive a tool call response, use the output to format an answer to the original user question.You are a helpful assistant with tool calling capabilities.'), ('human', ' Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.\n    Respond in the format {{"name": function name, "parameters": dictionary of argument name and its value}}.\n    Do not use variables.\n    {function}\n    {function_2}\n    Question: {input}')]
content='{"name": "oblicz_marze_detaliczna", "parameters": {"cena_hurtowa": 750.0}}' additional_kwargs={'usage': {'prompt_tokens': 482, 'completion_tokens': 30, 'total_tokens': 512}, 'stop_reason': 'stop', 'model_id': 'meta.llama3-70b-instruct-v1:0'} response_metadata={'usage': {'prompt_tokens': 482, 'completion_tokens': 30, 'total_tokens': 512}, 'stop_reason': 'stop', 'model_id': 'meta.llama3-70b-instruct-v1:0'} id='run-3ad78ad4-7aff-483a-9067-e7c82d23f475-0' usage

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent
from langchain_community.llms import AmazonAPIGateway
from langchain_community.agents.openai_assistant import OpenAIAssistantV2Runnable

tools = [calculate_retail_margin]
# agent = OpenAIAssistantV2Runnable.create_assistant(
#     name="langchain assistant",
#     instructions="Jesteś doradcą prawnym który ma odpowiadać na zadane pytania używając otrzymane narzędzia funkcje",
#     tools=tools,
#     model="gpt-4o",
# )
modelLlama = llm
prompt4o = ChatPromptTemplate.from_messages(
    [
        ("system",
         "Jesteś doradcą prawnym który ma odpowiadać na zadane pytania używając otrzymane narzędzia funkcje, zwróc uwagę na poprawność odpowiedzi i zastosowanie polskich znaków diakrytycznych"),
        ("human", "{input}"),
        # # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)
agent = create_tool_calling_agent(modelLlama, tools, prompt4o)

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)
kk = agent_executor.invoke(
    {"input": "Jaką marżę detaliczną należy zastosować dla leku, którego cena hurtowa wynosi 750 zł?"})
# kk = agent_executor.invoke({"input": "ile jest 2+2?"})

kk

{'input': 'Jaką marżę detaliczną należy zastosować dla leku, którego cena hurtowa wynosi 750 zł?',
 'output': 'W Polsce marża detaliczna dla leków jest uregulowana przepisami prawa. Zgodnie z art. 15 ust. 2 ustawy z dnia 6 września 2001 r. - Prawo farmaceutyczne (Dz. U. z 2021 r. poz. 1977), marża detaliczna dla leków nie może przekraczać 50% ceny hurtowej leku.\n\nW przypadku leku, którego cena hurtowa wynosi 750 zł, marża detaliczna nie może być wyższa niż 375 zł (50% z 750 zł).\n\nCena detaliczna leku powinna więc wynosić:\n\n750 zł (cena hurtowa) + 375 zł (marża detaliczna) = 1125 zł\n\nWarto zauważyć, że powyższa kalkulacja jest uproszczona i nie uwzględnia innych kosztów, które mogą być doliczone do ceny leku, takich jak podatek VAT czy koszty logistyki.'}